In [1]:
%env S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
%env PASSWORD_SQL=Urubu1@@
%env USER_SQL=urubu100

env: S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
env: PASSWORD_SQL=Urubu1@@
env: USER_SQL=urubu100


In [2]:
import os

S3_address = os.environ['S3ADRESS']
password_sql = os.environ['PASSWORD_SQL']
user_sql = os.environ['USER_SQL']

In [3]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.pandas as ps

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).config("spark.driver.memory", "15g").getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d3db570-8ed7-4d56-92de-949a60e4acdf;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 541ms :: artifacts dl 21ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [4]:
df = spark.read.option('delimiter', ',')\
    .option('header', 'true')\
    .csv(f'{S3_address}/dados-brutos/Value_of_Production_E_All_Data_(Normalized).csv')

22/06/07 02:24:34 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df = df.filter('Unit == "1000 Int. $"')

In [7]:
dataframe = df.filter('Item in ("Meat, cattle", "Soybeans")')

In [8]:
dataframe.select('Area').distinct().collect()

[Row(Area='Ethiopia PDR'),
 Row(Area='Germany'),
 Row(Area='China, Taiwan Province of'),
 Row(Area='Cambodia'),
 Row(Area='France'),
 Row(Area='Argentina'),
 Row(Area='Angola'),
 Row(Area='French Guyana'),
 Row(Area='Ecuador'),
 Row(Area='Albania'),
 Row(Area='Benin'),
 Row(Area='China'),
 Row(Area='Chile'),
 Row(Area='Bolivia (Plurinational State of)'),
 Row(Area='Croatia'),
 Row(Area='Czechoslovakia'),
 Row(Area='Burundi'),
 Row(Area='Gabon'),
 Row(Area='Czechia'),
 Row(Area='Bangladesh'),
 Row(Area='Bhutan'),
 Row(Area='Democratic Republic of the Congo'),
 Row(Area='China, mainland'),
 Row(Area="C�te d'Ivoire"),
 Row(Area='Georgia'),
 Row(Area='Azerbaijan'),
 Row(Area='Ethiopia'),
 Row(Area='Canada'),
 Row(Area='Belize'),
 Row(Area='Brazil'),
 Row(Area='Bosnia and Herzegovina'),
 Row(Area='Australia'),
 Row(Area='Cameroon'),
 Row(Area='Bulgaria'),
 Row(Area='Austria'),
 Row(Area='Costa Rica'),
 Row(Area='Egypt'),
 Row(Area='El Salvador'),
 Row(Area='Burkina Faso'),
 Row(Area='Colomb

In [9]:
dataframe_country = \
dataframe[dataframe.Area.isin('India', 'China', 'United States of America', 'Brazil', 'Argentina')]

In [10]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType, FloatType

In [11]:
dataframe_country = dataframe_country.withColumn('Item', when(dataframe_country.Item == 'Soybeans', 'Soja')
                             .when(dataframe_country.Item == 'Meat, cattle', "Carne bovina")
                             .otherwise(col('Item')))

In [12]:
dataframe_country = dataframe_country.withColumn('Area',when(dataframe_country.Area == 'United States of America', "EUA")
                             .when(dataframe_country.Area == 'Brazil', "Brasil")
                             .otherwise(col('Area')))

In [13]:
dataframe_country = dataframe_country.withColumn('Value', col('Value').cast(FloatType())) 
dataframe_country = dataframe_country.groupBy(['Year', 'Area', 'Item']).sum("Value").withColumnRenamed('sum(Value)', 'Value')

In [14]:
dataframe_country.show()

+----+---------+----+-----------+
|Year|     Area|Item|      Value|
+----+---------+----+-----------+
|2017|Argentina|Soja|2.1024684E7|
|1978|Argentina|Soja|   956161.0|
|1980|Argentina|Soja|  1338625.0|
|1994|Argentina|Soja|  4482443.0|
|1971|   Brasil|Soja|   794490.0|
|1983|Argentina|Soja|  1529857.0|
|1989|Argentina|Soja|  2486018.0|
|2005|Argentina|Soja|1.4644459E7|
|1968|    China|Soja|  3102931.0|
|2000|    China|Soja|  5893505.0|
|1975|Argentina|Soja|   185495.0|
|1993|Argentina|Soja|  4224471.0|
|1966|   Brasil|Soja|   227557.0|
|1981|   Brasil|Soja|  5739782.0|
|1971|    China|Soja|  3316344.0|
|1979|Argentina|Soja|  1415118.0|
|1985|    China|Soja|  4020545.0|
|1976|Argentina|Soja|   265813.0|
|2009|Argentina|Soja|1.1853861E7|
|1962|Argentina|Soja|     4291.0|
+----+---------+----+-----------+
only showing top 20 rows



In [15]:

dataframe_formated = dataframe_country[['Area','Item','Year','Value']]
_df = dataframe_formated.select(
    col('Area').alias('AREA'),
    col('Item').alias('ITEM'),
    col('Year').cast(IntegerType()).alias('YEAR'),
    col('Value').cast(FloatType()).alias('VALUE')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-tratados/world_meat_soybean_data')

22/06/07 02:25:14 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 02:25:23 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [16]:
dataframe = spark.read\
            .csv(f"{S3_address}/dados-tratados/world_meat_soybean_data")

In [17]:
dataframe = dataframe.withColumnRenamed("_c0", "pais")
dataframe = dataframe.withColumnRenamed("_c1", "item")
dataframe = dataframe.withColumnRenamed("_c2", "ano")
dataframe = dataframe.withColumnRenamed("_c3", "valor")

In [18]:
dataframe = dataframe.withColumn('valor', col('valor').cast(FloatType())) 
dataframe = dataframe.groupBy(['ano', 'pais', 'item']).sum("valor").withColumnRenamed('sum(valor)', 'valor')
dataframe.count()

295

In [19]:
dataframe.filter('ano == 2018').show()

+----+---------+----+-----------+
| ano|     pais|item|      valor|
+----+---------+----+-----------+
|2018|    India|Soja|  4181470.0|
|2018|   Brasil|Soja|  4.50973E7|
|2018|    China|Soja|  6108530.0|
|2018|Argentina|Soja|1.4452532E7|
|2018|      EUA|Soja|4.6092488E7|
+----+---------+----+-----------+



In [20]:
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType

_df = dataframe.select(
    col('pais').alias('pais'),
    col('valor').alias('valor'),
    col('item').alias('produto'),
    col('ano').cast(IntegerType()).alias('ano')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-clientes/world_meat_soybean_data')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://poupay-01.database.windows.net;databaseName=poupay;') \
 .option('dbtable', 'pais_producao') \
 .option('user', user_sql ) \
 .option('password', password_sql) \
 .save()

22/06/07 02:25:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 02:25:28 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
